In [1]:
import numpy as np
import matchzoo as mz

/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8457.68it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4591.87it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2697710.53it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8388.11it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4623.86it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 79215.55it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8031.41it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4764.18it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 84874.12it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9071.62it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4612.53it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 16596.30it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.ArcII()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [3, 3]
model.params['pool_2d_size'] = [[3, 3], [3, 3]]
model.params['optimizer'] = 'adam'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to ArcII.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002200     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 32)       28832       embedding[0][

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))
len(pred_y)

2341

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 14s 142ms/step - loss: 0.5740
Validation: normalized_discounted_cumulative_gain@3(0): 0.5993770560131093 - normalized_discounted_cumulative_gain@5(0): 0.650594665697349 - mean_average_precision(0): 0.6051205709433557
Epoch 2/30
102/102 [==============================] - 13s 126ms/step - loss: 0.1524
Validation: normalized_discounted_cumulative_gain@3(0): 0.5567182600732954 - normalized_discounted_cumulative_gain@5(0): 0.6180917374441505 - mean_average_precision(0): 0.5654610242446931
Epoch 3/30
102/102 [==============================] - 13s 124ms/step - loss: 0.0343
Validation: normalized_discounted_cumulative_gain@3(0): 0.5913961064490625 - normalized_discounted_cumulative_gain@5(0): 0.6537170030449837 - mean_average_precision(0): 0.6101269001399305
Epoch 4/30
102/102 [==============================] - 13s 126ms/step - loss: 0.0149
Validation: normalized_discounted_cumulative_gain@3(0): 0.5717007371002853 - normalized_discounted_cu

102/102 [==============================] - 13s 126ms/step - loss: 0.0153
Validation: normalized_discounted_cumulative_gain@3(0): 0.566055820213695 - normalized_discounted_cumulative_gain@5(0): 0.6283494205190083 - mean_average_precision(0): 0.5831118521198323
Epoch 30/30
102/102 [==============================] - 13s 123ms/step - loss: 0.0054
Validation: normalized_discounted_cumulative_gain@3(0): 0.5712552689173427 - normalized_discounted_cumulative_gain@5(0): 0.6259786277674115 - mean_average_precision(0): 0.5751640592363443
